## Generate scripts for ptycho reconstructions
1. `runPtycho.m` Matlab script
2. `4hr_2task_V100_runPtycho_01.sbatch` Slurm sbatch script for Campus Cluster
3. `BatchSubmit.sh` Shell script for submitting job file in batch with Linux

Chia-Hao Lee, chiahao3@illinois.edu

Last update: 2022/12/13

In [1]:
# Import package
import os

# 0. Setup the output folder and type in the input folders

In [2]:
output_dir = 'job_20221213_Themis_1212_heated_tBL-WSe2/'
if os.path.exists(output_dir)==False:
        os.makedirs(output_dir)
        print(f'{output_dir} has been created!')

In [3]:
# Manually select the folder based on the process priority and sbatch group
'''
scan_folder_list = [
    '2250_step128_20.5Mx_cl185mm_18mrad_df0_35pA_025C',
    '2252_step128_20.5Mx_cl185mm_18mrad_df+10_35pA_025C',
    '2253_step128_20.5Mx_cl185mm_18mrad_df+20_35pA_025C',
    '2317_step128_20.5Mx_cl185mm_18mrad_df0_35pA_300C',
    '2318_step128_20.5Mx_cl185mm_18mrad_df+15_35pA_300C',
    '2350_step128_20.5Mx_cl185mm_18mrad_df+15_35pA_400C',
    '2256_step64_20.5Mx_cl185mm_18mrad_df+15_35pA_025C',
    '2313_step64_20.5Mx_cl185mm_18mrad_df0_35pA_300C',
    '2314_step64_20.5Mx_cl185mm_18mrad_df0_35pA_300C',
    '2315_step64_20.5Mx_cl185mm_18mrad_df+15_35pA_300C',
    '2320_step64_20.5Mx_cl185mm_18mrad_df+20_35pA_300C',
    '2342_step64_20.5Mx_cl185mm_18mrad_df0_35pA_400C',
    '2343_step64_20.5Mx_cl185mm_18mrad_df+10_35pA_400C',
    '2345_step64_20.5Mx_cl185mm_18mrad_df0_35pA_400C',
    '2346_step64_20.5Mx_cl185mm_18mrad_df+15_35pA_400C',
    '2348_step64_20.5Mx_cl185mm_18mrad_df+15_35pA_400C',
    '2405_step64_20.5Mx_cl185mm_18mrad_df0_35pA_500C',
    '2407_step64_20.5Mx_cl185mm_18mrad_df+15_35pA_500C',
    '2408_step64_20.5Mx_cl185mm_18mrad_df+15_35pA_500C',
    '2409_step64_20.5Mx_cl185mm_18mrad_df+15_35pA_500C'
]
'''

scan_folder_list = [
    '2325_step64_20.5Mx_cl185mm_18mrad_df0_35pA_tBL_300C',
    '2327_step64_20.5Mx_cl185mm_18mrad_df+15_35pA_tBL_300C',
    '2330_step64_20.5Mx_cl185mm_18mrad_df+15_35pA_tBL_300C',
    '2332_step64_20.5Mx_cl185mm_18mrad_df+20_35pA_tBL_300C',
    '2353_step64_20.5Mx_cl185mm_18mrad_df0_35pA_tBL_400C',
    '2354_step64_20.5Mx_cl185mm_18mrad_df+15_35pA_tBL_400C',
    '2355_step128_20.5Mx_cl185mm_18mrad_df+15_35pA_tBL_400C',
    '2357_step128_20.5Mx_cl185mm_18mrad_df0_35pA_tBL_400C',
    '2415_step64_20.5Mx_cl185mm_18mrad_df0_35pA_tBL_500C',
    '2417_step64_20.5Mx_cl185mm_18mrad_df0_35pA_tBL_500C',
    '2418_step64_20.5Mx_cl185mm_18mrad_df+15_35pA_tBL_500C',
    '2419_step64_20.5Mx_cl185mm_18mrad_df+15_35pA_tBL_500C',
    '2420_step128_20.5Mx_cl185mm_18mrad_df+15_35pA_tBL_500C',
    '2423_step128_20.5Mx_cl185mm_18mrad_df+15_35pA_tBL_500C',
    '2421_step64_20.5Mx_cl185mm_18mrad_df0_35pA_tBL_500C',
    '2422_step64_20.5Mx_cl185mm_18mrad_df+15_35pA_tBL_500C',
    '2452_step64_20.5Mx_cl185mm_18mrad_df0_35pA_tBL_025C',
    '2453_step64_20.5Mx_cl185mm_18mrad_df+15_35pA_tBL_025C',
    '2454_step128_20.5Mx_cl185mm_18mrad_df0_35pA_tBL_025C',
    '2456_step128_20.5Mx_cl185mm_18mrad_df+15_35pA_tBL_025C'
]


print(f'Generating script files for {len(scan_folder_list)} EMPAD scans')

Generating script files for 20 EMPAD scans


In [4]:
# Preview the splitted the folder strings for experiment parameters
scan_folder_list[0].split("_")

['2325', 'step64', '20.5Mx', 'cl185mm', '18mrad', 'df0', '35pA', 'tBL', '300C']

In [5]:
# Scan independent values

exp_dir = '/scratch/users/chiahao3/20221212_TEM_ThemisZ_80kV_heated_WSe2_HomoH_EMPAD/' #change this for different experiment
ExpDate = '1212'
Sample = 'WSe2'

Np_crop_pad = [128,128] # size of diffraction patterns / probe used during reconstruction. can crop/pad to 64/256
resample_factor = 1 #DP upsample factor
scan_number = 1 #Ptychoshelves needs

# Microscope and acquisition parameters
ADU = 151 # Depends on kV, calibrated from the orignal EMPAD paper: doi:10.1017/S1431927615015664, 80kV: 151, 200kV: 393
voltage = 80
alpha0 = 18.44 #convergence semi angle in mrad # 18->18.44, 25.2->24.92, 36->36.4, 54->55.45
rbf = 8.57 #radius of central disk in px at given camera length. 16 - cl 230 mm; 
scan_step_size = 0.297 #For 128x128 scan positions, 1.182 - 5.1 Mx; 0.839 - 7.2 Mx; 0.605 - 10 Mx; 0.429 - 14.5Mx; 0.297 - 20.5 Mx; 0.208 - 29 Mx; 0.147 - 41 Mx %angstrom

# Reconstruction parameters
Niter = 9000 
Nprobe = 10 # # of probe modes
variable_probe_modes = 2 # possible # of modes for variable probe correction
grouping = 128 # group size. small -> better convergence but longer time/iteration
GPU_solver = 'MLs' # choose GPU solver: DM, ePIE, hPIE, MLc, Mls, -- recommended are MLc and MLs
errmetric = 'L1' # optimization likelihood - poisson, L1

In [6]:
# Scan dependent values

ExpTime_list, StepNum_list, StepSize_list, alpha0_list, rbf_list, df_list = [], [], [], [], [], []

for scan_folder in scan_folder_list:
    splitted_str = scan_folder.split("_")

    StepNum = int(splitted_str[1].replace('step', ''))

    ExpTime_list.append(splitted_str[0])
    StepNum_list.append(StepNum)
    StepSize_list.append(128//StepNum*scan_step_size)
    alpha0_list.append(alpha0)
    rbf_list.append(rbf)
    df_list.append(int(splitted_str[5].replace('df', ''))) #df are in UI-displayed nm
    

### (Optional) From matlab script template generate python code lines

In [ ]:
'''
template_name = 'runPtycho_cc_template.m'
with open(template_name, 'r+', encoding='utf-8') as template:
    lines = template.readlines()

with open('temporary.py', 'w', encoding='utf-8', newline='\n') as temporary:
    for line in lines:
        line = line.strip('\n')
        writing_line = f'MatlabScript.writelines([r"{line}' + r'", "\n"])'
        temporary.writelines([writing_line, '\n'])
    print('temporary.py has been created!')
'''
    

After generating the 'temporary.py', use a text editor to copy and paste the content to the following cell.

Note: To actually run this in Batch, the variable placeholders would need to be updated accordingly

## 1. runPtycho.m matlab script file

In [7]:
for ii, scan_folder in enumerate(scan_folder_list):

    MatlabScriptName = f'runPtycho_cc_Themis_{ExpDate}_{Sample}_{ExpTime_list[ii]}.m'
    with open(os.path.join(output_dir, MatlabScriptName), 'w', newline='\n') as MatlabScript:
        MatlabScript.writelines([r"% This script 1) prepares experimental electron ptycho. data for PtychoShelves", "\n"])
        MatlabScript.writelines([r"% and 2) run the electron ptychographic reconstruction. It's a combination of", "\n"])
        MatlabScript.writelines([r"% sample scripts from Yi Jiang's github repo: fold_slice ", "\n"])
        MatlabScript.writelines([r"% (https://github.com/yijiang1/fold_slice)", "\n"])
        MatlabScript.writelines([r"% Last modified by Chia-Hao Lee@UIUC, 2022/12/13", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% ", "\n"])
        MatlabScript.writelines([r"%%%%%%%%%%%%%%% Part I: Prepare the data and the initial probe %%%%%%%%%%%%", "\n"])
        MatlabScript.writelines([r"%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%", "\n"])
        MatlabScript.writelines([r"%% Step 0: Add path, clear variables", "\n"])
        MatlabScript.writelines([r"% Set paths so that the packpages can be found without changing working", "\n"])
        MatlabScript.writelines([r"% directories (i.e. run scripts elsewhere)", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"cSAXS_matlab_path = '/home/chiahao3/fold_slice';", "\n"])
        MatlabScript.writelines([r"ptycho_matlab_path = '/home/chiahao3/fold_slice/ptycho';", "\n"])
        MatlabScript.writelines([r"addpath(cSAXS_matlab_path);", "\n"])
        MatlabScript.writelines([r"addpath(ptycho_matlab_path);", "\n"])
        MatlabScript.writelines([r"addpath(strcat(ptycho_matlab_path,'/utils'));", "\n"])
        MatlabScript.writelines([r"addpath(strcat(ptycho_matlab_path,'/utils_electron'));", "\n"])
        MatlabScript.writelines([r"gpuDevice", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"%% Step 1: Setup parameters", "\n"])
        MatlabScript.writelines([r"% Data import and probe generation", "\n"])
        MatlabScript.writelines([r"exp_dir = '", exp_dir, r"'; %change this for different experiment", "\n"])
        MatlabScript.writelines([r"scan_dir = '", scan_folder, r"'; % change this for different scans", "\n"])
        MatlabScript.writelines([r"base_path = strcat(exp_dir, scan_dir, '/'); ", "\n"])
        MatlabScript.writelines([r"jobID = strcat('", Sample, r"_', scan_dir); %This will be used for the filename of param.m", "\n"])
        MatlabScript.writelines([r"dim_x = 128; %EMPAD pixel number", "\n"])
        MatlabScript.writelines([r"dim_y = 130; %Extra rows of EMPAD, will crop later", "\n"])
        MatlabScript.writelines([r"N_scan_x = ", str(StepNum_list[ii]), r"; %Real space scan positions", "\n"])
        MatlabScript.writelines([r"N_scan_y = ", str(StepNum_list[ii]), r";", "\n"])
        MatlabScript.writelines([r"dScanX = 1; %scan downsample factor, 2 is to load every other scan position", "\n"])
        MatlabScript.writelines([r"dScanY = 1; %scan downsample factor, 2 is to load every other scan position ", "\n"])
        MatlabScript.writelines([r"Np_crop_pad = [", str(Np_crop_pad[0]), ',', str(Np_crop_pad[0]), r"]; % size of diffraction patterns / probe used during reconstruction. can crop/pad to 64/256", "\n"])
        MatlabScript.writelines([r"resample_factor = ", str(resample_factor), r"; %DP upsample factor", "\n"])
        MatlabScript.writelines([r"scan_number = ",str(scan_number), r"; %Ptychoshelves needs", "\n"])
        MatlabScript.writelines([r"final_scan = [N_scan_x/dScanX, N_scan_y/dScanY]; %Final number of scan positions", "\n"])
        MatlabScript.writelines([r"final_dp_size = round(resample_factor * Np_crop_pad); %Final dimension of diffraction pattern", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"% Microscope and acquisition parameters", "\n"])
        MatlabScript.writelines([r"df = ", str(df_list[ii]), r"*10*-1*1.43; %defocus in angstrom, the sign is oppisite to UI defocus, and there's a 1.43 factor for actual defocus at 80kV for Themis", "\n"])
        MatlabScript.writelines([r"ADU = ", str(ADU), r"; % Depends on kV, calibrated from the orignal EMPAD paper: doi:10.1017/S1431927615015664, 80kV: 151, 200kV: 393", "\n"])
        MatlabScript.writelines([r"voltage = ", str(voltage), r";", "\n"])
        MatlabScript.writelines([r"alpha0 = ", str(alpha0_list[ii]), r"; %convergence semi angle # 18->18.44, 25.2->24.92, 36->36.4, 54->55.45", "\n"])
        MatlabScript.writelines([r"rbf = ", str(rbf_list[ii]), r"; %radius of central disk in px at given camera length. 16 - cl 230 mm; ", "\n"])
        MatlabScript.writelines([r"rot_ang = 0; %angle between cbed and scan coord.", "\n"])
        MatlabScript.writelines([r"scan_step_size = ", str(StepSize_list[ii]), r"; % For 128x128 scan positions, 1.182 - 5.1 Mx; 0.839 - 7.2 Mx; 0.605 - 10 Mx; 0.429 - 14.5Mx; 0.297 - 20.5 Mx; 0.208 - 29 Mx; 0.147 - 41 Mx %angstrom", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"% Reconstruction parameters", "\n"])
        MatlabScript.writelines([r"data_descriptor = '", Sample, r"_'; %A short string that describe data when sending notifications", "\n"])
        MatlabScript.writelines([r"Niter = ", str(Niter), r";", "\n"])
        MatlabScript.writelines([r"Niter_save_results = 100;", "\n"])
        MatlabScript.writelines([r"Niter_plot_results = 100;", "\n"])
        MatlabScript.writelines([r"Nprobe = ", str(Nprobe), r"; % # of probe modes", "\n"])
        MatlabScript.writelines([r"variable_probe_modes = ", str(variable_probe_modes), r"; % 2 if possible # of modes for variable probe correction", "\n"])
        MatlabScript.writelines([r"grouping = ", str(grouping), r"; % group size. small -> better convergence but longer time/iteration", "\n"])
        MatlabScript.writelines([r"N_pos_corr = 0; % iteration number to start position correction. inf means no position correction", "\n"])
        MatlabScript.writelines([r"GPU_solver = '", GPU_solver, r"'; % choose GPU solver: DM, ePIE, hPIE, MLc, Mls, -- recommended are MLc and MLs", "\n"])
        MatlabScript.writelines([r"errmetric = '", errmetric, r"';            % optimization likelihood - poisson, L1", "\n"])
        MatlabScript.writelines([r"recon_time = datestr(now, 'yyyy-mm-dd_HH-MM-SS');", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"initial_probe_file = strcat(base_path, sprintf('%i/init_probe_%i.mat', scan_number, final_dp_size(1)));", "\n"])
        MatlabScript.writelines([r"%init_recon_file = '/scratch/users/chiahao3/20220712_TEM_Talos_WSe2_16_BL_EMPAD/0032_WSe2_bi_10.5mrad_cl_205_28.5mx_def0_1/1/roi1_Ndp128_step128/MLs_L1_p10_g128pc0_noModel_updW100_vp2_vi_mm_dpFlip_T/Niter2800.mat';", "\n"])
        MatlabScript.writelines([r"use_previous_probe = false; %set false when you use a new data set", "\n"])
        MatlabScript.writelines([r"use_previous_object = false; %false for new data", "\n"])
        MatlabScript.writelines([r"use_previous_position = false; % false for new data", "\n"])
        MatlabScript.writelines([r"probe_change_start = 1;      % Start updating probe at this iteration number", "\n"])
        MatlabScript.writelines([r"object_change_start = 1;       % Start updating object at this iteration number", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"% Flags", "\n"])
        MatlabScript.writelines([r"if Np_crop_pad(1)~=128", "\n"])
        MatlabScript.writelines([r"    if_crop_pad = true;", "\n"])
        MatlabScript.writelines([r"else", "\n"])
        MatlabScript.writelines([r"    if_crop_pad = false;   ", "\n"])
        MatlabScript.writelines([r"end", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"if resample_factor~=1", "\n"])
        MatlabScript.writelines([r"    if_resample = true;", "\n"])
        MatlabScript.writelines([r"    rbf = rbf * resample_factor; ", "\n"])
        MatlabScript.writelines([r"else", "\n"])
        MatlabScript.writelines([r"    if_resample = false;", "\n"])
        MatlabScript.writelines([r"end", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"save(strcat(base_path, jobID, sprintf('_params_%s.mat', recon_time))); %Save the parameters", "\n"])
        MatlabScript.writelines([r"disp(sprintf('Started at %s', recon_time))", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% ", "\n"])
        MatlabScript.writelines([r"%%%%%%%%%%%%%%% Shouldn't need to change anything below for common usage %%", "\n"])
        MatlabScript.writelines([r"%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"%% Step 2: load data", "\n"])
        MatlabScript.writelines([r"scan_name = sprintf('scan_x%i_y%i.raw', N_scan_x, N_scan_y);", "\n"])
        MatlabScript.writelines([r"fin = fread(fopen(strcat(base_path, scan_name),'r'), dim_x * dim_y * N_scan_x * N_scan_y,'float32=>float32'); % Load as 1-D vector with float32 and save as float32", "\n"])
        MatlabScript.writelines([r"fin_reshape = reshape(fin, dim_x, dim_y, N_scan_x, N_scan_y); %Matlab fill its column first, so the DP and scan pattern would be automatically transposed", "\n"])
        MatlabScript.writelines([r"cbed = double(fin_reshape(1:128,1:128,1:dScanX:end,1:dScanY:end)); % crop the CBED into 128x128", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"%% Step 3: Pre-process the size of the diffraction patterns (pad, crop, resample)", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"[ndpx, ndpy, npx, npy] = size(cbed); % get the raw data dimentsion", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"% Step 3-1: Crop / pad the cbed based on Np_crop_pad", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"if ndpy < Np_crop_pad(1) % pad zeros", "\n"])
        MatlabScript.writelines([r"    dp=padarray(cbed,[(Np_crop_pad(1)-ndpy)/2,(Np_crop_pad(2)-ndpx)/2,0,0],0,'both');", "\n"])
        MatlabScript.writelines([r"else", "\n"])
        MatlabScript.writelines([r"    dp=crop_pad(cbed, Np_crop_pad);", "\n"])
        MatlabScript.writelines([r"end", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"% Step 3-2: Resample the cbed based on resample_factor", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"if resample_factor ~= 1", "\n"])
        MatlabScript.writelines([r"    dp = reshape(dp, Np_crop_pad(1), Np_crop_pad(2), []);%reshape to 3D so that we can use the built-in imresize3 instead of imresizen", "\n"])
        MatlabScript.writelines([r"    dp = imresize3(dp, [final_dp_size(1), final_dp_size(2), npx*npy]);", "\n"])
        MatlabScript.writelines([r"else", "\n"])
        MatlabScript.writelines([r"	dp = reshape(dp, Np_crop_pad(1), Np_crop_pad(2), []);", "\n"])
        MatlabScript.writelines([r"end", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"% Normalizing the cbed intensity", "\n"])
        MatlabScript.writelines([r"dp = dp / ADU; % convert to electron count", "\n"])
        MatlabScript.writelines([r"Itot=mean(squeeze(sum(sum(dp,1),2))); %need this for normalizting initial probe", "\n"])
        MatlabScript.writelines([r"dp=reshape(dp, final_dp_size(1), final_dp_size(2), npx, npy);", "\n"])
        MatlabScript.writelines([r"[Ndpx, Ndpy, Npx, Npy] = size(dp); %get the final data dimension", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"% calculate pixel size (1/A) in diffraction plane", "\n"])
        MatlabScript.writelines([r"[~,lambda]=electronwavelength(voltage);", "\n"])
        MatlabScript.writelines([r"dk=alpha0/1e3/rbf/lambda; %%% PtychoShelves script needs this %%%", "\n"])
        MatlabScript.writelines([r"dp=reshape(dp, Ndpx, Ndpy, []); %PtychoShelves actually takes this 3D shape", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"%% Step 4: save 4D-STEM in a .hdf5 file (needed by Ptychoshelves)", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"save_dir = strcat(base_path,num2str(scan_number),'/');", "\n"])
        MatlabScript.writelines([r"mkdir(save_dir)", "\n"])
        MatlabScript.writelines([r"roi_label = sprintf('%i_Ndp%i_step%i', scan_number, Ndpx, Npx);", "\n"])
        MatlabScript.writelines([r"saveName = strcat('data_roi',roi_label,'_dp.hdf5');", "\n"])
        MatlabScript.writelines([r"dp_dir = strcat(save_dir,saveName);", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"if isfile(dp_dir)", "\n"])
        MatlabScript.writelines([r"    delete(dp_dir);", "\n"])
        MatlabScript.writelines([r"end    ", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"h5create(dp_dir, '/dp', size(dp),'ChunkSize',[size(dp,1), size(dp,2), 1],'Deflate',4)", "\n"])
        MatlabScript.writelines([r"h5write(dp_dir, '/dp', dp)", "\n"])
        MatlabScript.writelines([r"disp(strcat(saveName, ' has been saved!'))", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"%% Step 5: prepare an save initial probe", "\n"])
        MatlabScript.writelines([r"dx=1/Ndpx/dk; %% pixel size in real space (angstrom)", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"par_probe = {};", "\n"])
        MatlabScript.writelines([r"par_probe.df = df;", "\n"])
        MatlabScript.writelines([r"par_probe.voltage = voltage;", "\n"])
        MatlabScript.writelines([r"par_probe.alpha_max = alpha0;", "\n"])
        MatlabScript.writelines([r"par_probe.plotting = true;", "\n"])
        MatlabScript.writelines([r"probe = make_tem_probe(dx, Ndpx, par_probe);", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"probe=probe/sqrt(sum(sum(abs(probe.^2))))*sqrt(Itot)/sqrt(Ndpx*Ndpy);", "\n"])
        MatlabScript.writelines([r"probe=single(probe);", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"% add parameters for PtychoShelves", "\n"])
        MatlabScript.writelines([r"p = {};", "\n"])
        MatlabScript.writelines([r"p.binning = false;", "\n"])
        MatlabScript.writelines([r"p.detector.binning = false;", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"% save initial probe", "\n"])
        MatlabScript.writelines([r"save(strcat(save_dir,sprintf('/init_probe_%i.mat', Ndpx)),'probe','p')", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"%% %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%", "\n"])
        MatlabScript.writelines([r"%%%%%%%%%%%%%%% Part II: Prepare and run the reconstruction %%%%%%%%%%%%%%%", "\n"])
        MatlabScript.writelines([r"%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"%% Step 6: initialize data parameters", "\n"])
        MatlabScript.writelines([r"p = struct();", "\n"])
        MatlabScript.writelines([r"p.   verbose_level = 3;                            % verbosity for standard output (0-1 for loops, 2-3 for testing and adjustments, >= 4 for debugging)", "\n"])
        MatlabScript.writelines([r"p.   use_display = Niter_plot_results< Niter;                                      % global switch for display, if [] then true for verbose > 1", "\n"])
        MatlabScript.writelines([r"p.   scan_number = scan_number;                                    % Multiple scan numbers for shared scans", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"% Geometry", "\n"])
        MatlabScript.writelines([r"p.   z = 1;                                             % Distance from object to detector. Always 1 for electron ptycho", "\n"])
        MatlabScript.writelines([r"p.   asize = [Ndpx,Ndpx];                                     % Diffr. patt. array size", "\n"])
        MatlabScript.writelines([r"p.   ctr = [fix(Ndpx/2)+1, fix(Ndpx/2)+1];                                       % Diffr. patt. center coordinates (y,x) (empty means middle of the array); e.g. [100 207;100+20 207+10];", "\n"])
        MatlabScript.writelines([r"p.   beam_source = 'electron';                         % Added by YJ for electron pty. Use relativistic corrected formula for wavelength. Also change the units on figures", "\n"])
        MatlabScript.writelines([r"%p.   dk = dk;                                          % Added by YJ. dk is the pixel size in cbed (1/A). This is used to determine pixel size in electron ptycho", "\n"])
        MatlabScript.writelines([r"p.   d_alpha = alpha0/rbf;                              % Added by YJ. d_alpha is the pixel size in cbed (mrad). This is used to determine pixel size in electron ptycho", "\n"])
        MatlabScript.writelines([r"p.   prop_regime = 'farfield';                              % propagation regime: nearfield, farfield (default), !! nearfield is supported only by GPU engines ", "\n"])
        MatlabScript.writelines([r"p.   focus_to_sample_distance = [];                         % sample to focus distance, parameter to be set for nearfield ptychography, otherwise it is ignored ", "\n"])
        MatlabScript.writelines([r"p.   energy = voltage;                                           % Energy (in keV), leave empty to use spec entry mokev", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"%p.   affine_angle = 0;                                     % Not used by ptycho_recons at all. This allows you to define a variable for the affine matrix below and keep it in p for future record. This is used later by the affine_matrix_search.m script", "\n"])
        MatlabScript.writelines([r"%p.   affine_matrix = [1 , 0; 0, 1] ; % Applies affine transformation (e.g. rotation, stretching) to the positions (ignore by = []). Convention [yn;xn] = M*[y;x].", "\n"])
        MatlabScript.writelines([r"affine_mat  = compose_affine_matrix(1, 0, rot_ang, 0);", "\n"])
        MatlabScript.writelines([r"p.   affine_matrix = affine_mat ; % Applies affine transformation (e.g. rotation, stretching) to the positions (ignore by = []). Convention [yn;xn] = M*[y;x].", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"% Scan meta data", "\n"])
        MatlabScript.writelines([r"p.   src_metadata = 'none';                                 % source of the meta data, following options are supported: 'spec', 'none' , 'artificial' - or add new to +scan/+meta/", "\n"])
        MatlabScript.writelines([r"p.   queue.lockfile = false;                                % If true writes a lock file, if lock file exists skips recontruction", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"% Data preparation", "\n"])
        MatlabScript.writelines([r"p.   detector.name = 'empad';                           % see +detectors/ folder ", "\n"])
        MatlabScript.writelines([r"p.   detector.check_2_detpos = [];                          % = []; (ignores)   = 270; compares to dettrx to see if p.ctr should be reversed (for OMNY shared scans 1221122), make equal to the middle point of dettrx between the 2 detector positions", "\n"])
        MatlabScript.writelines([r"p.   detector.data_prefix = '';                             % Default using current eaccount e.g. e14169_1_", "\n"])
        MatlabScript.writelines([r"p.   detector.binning = false;                              % = true to perform 2x2 binning of detector pixels, for binning = N do 2^Nx2^N binning", "\n"])
        MatlabScript.writelines([r"p.   detector.upsampling = false;                           % upsample the measured data by 2^data_upsampling, (transposed operator to the binning), it can be used for superresolution in nearfield ptychography or to account for undersampling in a far-field dataset", "\n"])
        MatlabScript.writelines([r"p.   detector.burst_frames = 1;                             % number of frames collected per scan position", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"p.   prepare.data_preparator = 'matlab_aps';                % data preparator; 'python' or 'matlab' or 'matlab_aps'", "\n"])
        MatlabScript.writelines([r"p.   prepare.auto_prepare_data = true;                      % if true: prepare dataset from raw measurements if the prepared data does not exist", "\n"])
        MatlabScript.writelines([r"p.   prepare.force_preparation_data = true;                 % Prepare dataset even if it exists, it will overwrite the file % Default: @prepare_data_2d", "\n"])
        MatlabScript.writelines([r"p.   prepare.store_prepared_data = false;                    % store the loaded data to h5 even for non-external engines (i.e. other than c_solver)", "\n"])
        MatlabScript.writelines([r"p.   prepare.prepare_data_function = '';                    % (used only if data should be prepared) custom data preparation function handle;", "\n"])
        MatlabScript.writelines([r"p.   prepare.auto_center_data = false;                      % if matlab data preparator is used, try to automatically center the diffraction pattern to keep center of mass in center of diffraction", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"p.   src_positions = 'matlab_pos';                           % 'spec', 'orchestra', 'load_from_file', 'matlab_pos' (scan params are defined below) or add new position loaders to +scan/+positions/", "\n"])
        MatlabScript.writelines([r"p.   positions_file = [''];    %Filename pattern for position files, Example: ['../../specES1/scan_positions/scan_%05d.dat']; (the scan number will be automatically filled in)", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"if use_previous_position", "\n"])
        MatlabScript.writelines([r"    p.   scan.type = 'custom_GPU';     ", "\n"])
        MatlabScript.writelines([r"    p.   scan.custom_positions_source = init_recon_file; % custom: a string name of a function that defines the positions; also accepts mat file with entry 'pos', see +scans/+positions/+mat_pos.m", "\n"])
        MatlabScript.writelines([r"else", "\n"])
        MatlabScript.writelines([r"    % scan parameters for option src_positions = 'matlab_pos';", "\n"])
        MatlabScript.writelines([r"    p.   scan.type = 'raster';                                  % {'round', 'raster', 'round_roi', 'custom'}", "\n"])
        MatlabScript.writelines([r"    p.   scan.custom_positions_source = '';", "\n"])
        MatlabScript.writelines([r"end", "\n"])
        MatlabScript.writelines([r"p.   scan.roi_label = roi_label;                            % For APS data", "\n"])
        MatlabScript.writelines([r"p.   scan.format = '%01d';                      % For APS data format for scan directory generation", "\n"])
        MatlabScript.writelines([r"p.   scan.radius_in = 0;                                    % round scan: interior radius of the round scan", "\n"])
        MatlabScript.writelines([r"p.   scan.radius_out = 5e-6;                                % round scan: exterior radius of the round scan", "\n"])
        MatlabScript.writelines([r"p.   scan.nr = 10;                                          % round scan: number of intervals (# of shells - 1)", "\n"])
        MatlabScript.writelines([r"p.   scan.nth = 3;                                          % round scan: number of points in the first shell", "\n"])
        MatlabScript.writelines([r"p.   scan.lx = 20e-6;                                       % round_roi scan: width of the roi", "\n"])
        MatlabScript.writelines([r"p.   scan.ly = 20e-6;                                       % round_roi scan: height of the roi", "\n"])
        MatlabScript.writelines([r"p.   scan.dr = 1.5e-6;                                      % round_roi scan: shell step size", "\n"])
        MatlabScript.writelines([r"p.   scan.nx = Npx;      %size(dp,3)                                  % raster scan: number of steps in x", "\n"])
        MatlabScript.writelines([r"p.   scan.ny = Npy;                                                   % raster scan: number of steps in y", "\n"])
        MatlabScript.writelines([r"p.   scan.step_size_x = dScanX*scan_step_size;                               % raster scan: step size (grid spacing)", "\n"])
        MatlabScript.writelines([r"p.   scan.step_size_y = dScanY*scan_step_size;                               % raster scan: step size (grid spacing)", "\n"])
        MatlabScript.writelines([r"p.   scan.custom_flip = [1,1,0];                            % raster scan: apply custom flip [fliplr, flipud, transpose] to positions- similar to eng.custom_data_flip in GPU engines. Added by ZC.", "\n"])
        MatlabScript.writelines([r"p.   scan.step_randn_offset = 0;                            % raster scan: relative random offset from the ideal periodic grid to avoid the raster grid pathology ", "\n"])
        MatlabScript.writelines([r"p.   scan.b = 0;                                            % fermat: angular offset", "\n"])
        MatlabScript.writelines([r"p.   scan.n_max = 1e4;                                      % fermat: maximal number of points generated ", "\n"])
        MatlabScript.writelines([r"p.   scan.step = 0.5e-6;                                      % fermat: step size ", "\n"])
        MatlabScript.writelines([r"p.   scan.cenxy = [0,0];                                    % fermat: position of center offset ", "\n"])
        MatlabScript.writelines([r"p.   scan.roi = [];                                         % Region of interest in the object [xmin xmax ymin ymax] in meters. Points outside this region are not used for reconstruction.", "\n"])
        MatlabScript.writelines([r"                                                            %  (relative to upper corner for raster scans and to center for round scans)    ", "\n"])
        MatlabScript.writelines([r"                                                            % custom: a string name of a function that defines the positions; also accepts mat file with entry 'pos', see +scans/+positions/+mat_pos.m", "\n"])
        MatlabScript.writelines([r"p.   scan.custom_params = [];                               % custom: the parameters to feed to the custom position function.", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"% I/O", "\n"])
        MatlabScript.writelines([r"p.   prefix = '';                                              % For automatic output filenames. If empty: scan number", "\n"])
        MatlabScript.writelines([r"p.   suffix = strcat('ML_recon');              % Optional suffix for reconstruction ", "\n"])
        MatlabScript.writelines([r"p.   scan_string_format = '%01d';                  % format for scan string generation, it is used e.g for plotting and data saving ", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"%%%p.   base_path = '../../';                                  % base path : used for automatic generation of other paths ", "\n"])
        MatlabScript.writelines([r"p.   base_path = base_path;     % base path : used for automatic generation of other paths ", "\n"])
        MatlabScript.writelines([r"p.   specfile = '';                                         % Name of spec file to get motor positions and check end of scan, defaut is p.spec_file == p.base_path;", "\n"])
        MatlabScript.writelines([r"p.   ptycho_matlab_path = ptycho_matlab_path;               % cSAXS ptycho package path", "\n"])
        MatlabScript.writelines([r"p.   cSAXS_matlab_path = cSAXS_matlab_path;                 % cSAXS base package path", "\n"])
        MatlabScript.writelines([r"p.   raw_data_path{1} = '';                                 % Default using compile_x12sa_filename, used only if data should be prepared automatically", "\n"])
        MatlabScript.writelines([r"p.   prepare_data_path = '';                                % Default: base_path + 'analysis'. Other example: '/afs/psi.ch/project/CDI/cSAXS_project/analysis2/'; also supports %u to insert the scan number at a later point (e.g. '/afs/psi.ch/project/CDI/cSAXS_project/analysis2/S%.5u')", "\n"])
        MatlabScript.writelines([r"p.   prepare_data_filename = [];                            % Leave empty for default file name generation, otherwise use [sprintf('S%05d_data_%03dx%03d',p.scan_number(1), p.asize(1), p.asize(2)) p.prep_data_suffix '.h5'] as default ", "\n"])
        MatlabScript.writelines([r"p.   save_path{1} = '';                                     % Default: base_path + 'analysis'. Other example: '/afs/psi.ch/project/CDI/cSAXS_project/analysis2/'; also supports %u to insert the scan number at a later point (e.g. '/afs/psi.ch/project/CDI/cSAXS_project/analysis2/S%.5u')", "\n"])
        MatlabScript.writelines([r"p.   io.default_mask_file = '';                             % load detector mask defined in this file instead of the mask in the detector packages, (used only if data should be prepared) ", "\n"])
        MatlabScript.writelines([r"p.   io.default_mask_type = 'binary';                       % (used only if data should be prepared) ['binary', 'indices']. Default: 'binary' ", "\n"])
        MatlabScript.writelines([r"p.   io.file_compression = 0;                               % reconstruction file compression for HDF5 files; 0 for no compression", "\n"])
        MatlabScript.writelines([r"p.   io.data_compression = 3;                               % prepared data file compression for HDF5 files; 0 for no compression", "\n"])
        MatlabScript.writelines([r"p.   io.load_prep_pos = false;                              % load positions from prepared data file and ignore positions provided by metadata", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"p.   io.data_descriptor = data_descriptor;                     %added by YJ. A short string that describe data when sending notifications ", "\n"])
        MatlabScript.writelines([r"p.   io.phone_number = '';                      % phone number for sending messages", "\n"])
        MatlabScript.writelines([r"p.   io.send_failed_scans_SMS = false;                       % send message if p.queue_max_attempts is exceeded", "\n"])
        MatlabScript.writelines([r"p.   io.send_finished_recon_SMS = false;                    % send message after the reconstruction is completed", "\n"])
        MatlabScript.writelines([r"p.   io.send_crashed_recon_SMS = false;                     % send message if the reconstruction crashes", "\n"])
        MatlabScript.writelines([r"p.   io.SMS_sleep = 1800;                                   % max 1 message per SMS_sleep seconds", "\n"])
        MatlabScript.writelines([r"p.   io.script_name = mfilename;                             % added by YJ. store matlab script name", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"p.   artificial_data_file = 'template_artificial_data';     % artificial data parameters, set p.src_metadata = 'artificial' to use this template", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"%% Step 7: Initial reconstruction parameters", "\n"])
        MatlabScript.writelines([r"% Initial iterate object", "\n"])
        MatlabScript.writelines([r"if use_previous_object", "\n"])
        MatlabScript.writelines([r"    p.   model_object = false;                                   % Use model object, if false load it from file ", "\n"])
        MatlabScript.writelines([r"    p.   model.object_type = 'rand';                            % specify how the object shall be created; use 'rand' for a random initial guess; use 'amplitude' for an initial guess based on the prepared data", "\n"])
        MatlabScript.writelines([r"    p.   initial_iterate_object_file{1} = init_recon_file;                   %  use this mat-file as initial guess of object, it is possible to use wild characters and pattern filling, example: '../analysis/S%05i/wrap_*_1024x1024_1_recons*'", "\n"])
        MatlabScript.writelines([r"else", "\n"])
        MatlabScript.writelines([r"    p.   model_object = true;                                   % Use model object, if false load it from file ", "\n"])
        MatlabScript.writelines([r"    p.   model.object_type = 'rand';                            % specify how the object shall be created; use 'rand' for a random initial guess; use 'amplitude' for an initial guess based on the prepared data", "\n"])
        MatlabScript.writelines([r"    p.   initial_iterate_object_file{1} = '';                   %  use this mat-file as initial guess of object, it is possible to use wild characters and pattern filling, example: '../analysis/S%05i/wrap_*_1024x1024_1_recons*'", "\n"])
        MatlabScript.writelines([r"end", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"% Initial iterate probe", "\n"])
        MatlabScript.writelines([r"p.   model_probe = false;                                   % Use model probe, if false load it from file ", "\n"])
        MatlabScript.writelines([r"p.   model.probe_alpha_max = alpha0;                          % Modal STEM probe's aperture size", "\n"])
        MatlabScript.writelines([r"p.   model.probe_df = 0;                                 % Modal STEM probe's defocus", "\n"])
        MatlabScript.writelines([r"p.   model.probe_c3 = 0;                                    % Modal STEM probe's third-order spherical aberration in angstrom (optional)", "\n"])
        MatlabScript.writelines([r"p.   model.probe_c5 = 0;                                    % Modal STEM probe's fifth-order spherical aberration in angstrom (optional)", "\n"])
        MatlabScript.writelines([r"p.   model.probe_c7 = 0;                                    % Modal STEM probe's seventh-order spherical aberration in angstrom (optional)", "\n"])
        MatlabScript.writelines([r"p.   model.probe_f_a2 = 0;                                  % Modal STEM probe's twofold astigmatism in angstrom (optional)", "\n"])
        MatlabScript.writelines([r"p.   model.probe_theta_a2 = 0;                              % Modal STEM probe's twofold azimuthal orientation in radian (optional)", "\n"])
        MatlabScript.writelines([r"p.   model.probe_f_a3 = 0;                                  % Modal STEM probe's threefold astigmatism in angstrom (optional)", "\n"])
        MatlabScript.writelines([r"p.   model.probe_theta_a3 = 0;                              % Modal STEM probe's threefold azimuthal orientation in radian (optional)", "\n"])
        MatlabScript.writelines([r"p.   model.probe_f_c3 = 0;                                  % Modal STEM probe's coma in angstrom (optional)", "\n"])
        MatlabScript.writelines([r"p.   model.probe_theta_c3 = 0;                              % Modal STEM probe's coma azimuthal orientation in radian (optional)", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"if use_previous_probe", "\n"])
        MatlabScript.writelines([r"    p.   model_probe = false;", "\n"])
        MatlabScript.writelines([r"    p.   initial_probe_file = init_recon_file;", "\n"])
        MatlabScript.writelines([r"    p.   normalize_init_probe = false;                           % Added by YJ. Can be used to disable normalization of initial probes", "\n"])
        MatlabScript.writelines([r"    p.   ortho_probes = true;                                   % orthogonalize probes after each engine", "\n"])
        MatlabScript.writelines([r"else", "\n"])
        MatlabScript.writelines([r"    p.   initial_probe_file = initial_probe_file; ", "\n"])
        MatlabScript.writelines([r"    p.   ortho_probes = true;                                   % orthogonalize probes after each engine", "\n"])
        MatlabScript.writelines([r"end", "\n"])
        MatlabScript.writelines([r"p.   probe_file_propagation = 0.0e-3;                            % Distance for propagating the probe from file in meters, = 0 to ignore", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"% Shared scans - Currently working only for sharing probe and object", "\n"])
        MatlabScript.writelines([r"p.   share_probe  = 0;                                      % Share probe between scans. Can be either a number/boolean or a list of numbers, specifying the probe index; e.g. [1 2 2] to share the probes between the second and third scan. ", "\n"])
        MatlabScript.writelines([r"p.   share_object = 0;                                      % Share object between scans. Can be either a number/boolean or a list of numbers, specifying the object index; e.g. [1 2 2] to share the objects between the second and third scan. ", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"% Modes", "\n"])
        MatlabScript.writelines([r"p.   probe_modes  = Nprobe;                                 % Number of coherent modes for probe", "\n"])
        MatlabScript.writelines([r"p.   object_modes = 1;                                      % Number of coherent modes for object", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"% Mode starting guess", "\n"])
        MatlabScript.writelines([r"p.   mode_start_pow = 0.02;                               % Normalized intensity on probe modes > 1. Can be a number (all higher modes equal) or a vector", "\n"])
        MatlabScript.writelines([r"p.   mode_start = 'herm';                                   % (for probe) = 'rand', = 'herm' (Hermitian-like base), = 'hermver' (vertical modes only), = 'hermhor' (horizontal modes only)", "\n"])
        MatlabScript.writelines([r"p.   ortho_probes = true;                                   % orthogonalize probes after each engine", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"%% Step 8: initialize Plot, save and analyze parameters", "\n"])
        MatlabScript.writelines([r"p.   plot.prepared_data = false;                         % plot prepared data", "\n"])
        MatlabScript.writelines([r"p.   plot.interval = [];                                    % plot each interval-th iteration, does not work for c_solver code", "\n"])
        MatlabScript.writelines([r"p.   plot.log_scale = [0 0];                                % Plot on log scale for x and y", "\n"])
        MatlabScript.writelines([r"p.   plot.realaxes = true;                                  % Plots show scale in microns", "\n"])
        MatlabScript.writelines([r"p.   plot.remove_phase_ramp = false;                        % Remove phase ramp from the plotted / saved phase figures ", "\n"])
        MatlabScript.writelines([r"p.   plot.fov_box = false;                                   % Plot the scanning FOV box on the object (both phase and amplitude)", "\n"])
        MatlabScript.writelines([r"p.   plot.fov_box_color = 'r';                              % Color of the scanning FOV box", "\n"])
        MatlabScript.writelines([r"p.   plot.positions = true;                                 % Plot the scanning positions", "\n"])
        MatlabScript.writelines([r"p.   plot.mask_bool = true;                                 % Mask the noisy contour of the reconstructed object in plots", "\n"])
        MatlabScript.writelines([r"p.   plot.windowautopos = true;                             % First plotting will auto position windows", "\n"])
        MatlabScript.writelines([r"p.   plot.obj_apod = false;                                 % Apply apodization to the reconstructed object;", "\n"])
        MatlabScript.writelines([r"p.   plot.prop_obj = 0;                                     % Distance to propagate reconstructed object before plotting [m]", "\n"])
        MatlabScript.writelines([r"p.   plot.show_layers = true;                               % show each layer in multilayer reconstruction ", "\n"])
        MatlabScript.writelines([r"p.   plot.show_layers_stack = false;                        % show each layer in multilayer reconstruction by imagesc3D", "\n"])
        MatlabScript.writelines([r"p.   plot.object_spectrum = [];                             % Plot propagated object (FFT for conventional ptycho); if empty then default is false if verbose_level < 3 and true otherwise", "\n"])
        MatlabScript.writelines([r"p.   plot.probe_spectrum = [];                              % Plot propagated probe (FFT for conventional ptycho); if empty then default is false if verbose_level < 3 and true otherwise", "\n"])
        MatlabScript.writelines([r"p.   plot.conjugate = false;                                % plot complex conjugate of the reconstruction ", "\n"])
        MatlabScript.writelines([r"p.   plot.horz_fact = 2.5;                                  % Scales the space that the ptycho figures take horizontally", "\n"])
        MatlabScript.writelines([r"p.   plot.FP_maskdim = 180e-6;                              % Filter the backpropagation (Fourier Ptychography)", "\n"])
        MatlabScript.writelines([r"p.   plot.calc_FSC = false;                                 % Calculate the Fourier Shell correlation for 2 scans or compare with model in case of artificial data tests ", "\n"])
        MatlabScript.writelines([r"p.   plot.show_FSC = false;                                 % Show the FSC plots, including the cropped FOV", "\n"])
        MatlabScript.writelines([r"p.   plot.residua = false;                                  % highlight phase-residua in the image of the reconstructed phase", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"p.   save.external = true;                             % Use a new Matlab session to run save final figures (saves ~6s per reconstruction). Please be aware that this might lead to an accumulation of Matlab sessions if your single reconstruction is very fast.", "\n"])
        MatlabScript.writelines([r"p.   save.store_images = false;                              % Write preview images containing the final reconstructions in [p.base_path,'analysis/online/ptycho/'] if p.use_display = 0 then the figures are opened invisible in order to create the nice layout. It writes images in analysis/online/ptycho", "\n"])
        MatlabScript.writelines([r"p.   save.store_images_intermediate = false;                % save images to disk after each engine", "\n"])
        MatlabScript.writelines([r"p.   save.store_images_ids = 1:4;                           % identifiers  of the figure to be stored, 1=obj. amplitude, 2=obj. phase, 3=probes, 4=errors, 5=probes spectrum, 6=object spectrum", "\n"])
        MatlabScript.writelines([r"p.   save.store_images_format = 'png';                      % data type of the stored images jpg or png ", "\n"])
        MatlabScript.writelines([r"p.   save.store_images_dpi = 150;                           % DPI of the stored bitmap images ", "\n"])
        MatlabScript.writelines([r"p.   save.exclude = {'fmag', 'fmask', 'illum_sum'};         % exclude variables to reduce the file size on disk", "\n"])
        MatlabScript.writelines([r"p.   save.save_reconstructions_intermediate = false;        % save final object and probes after each engine", "\n"])
        MatlabScript.writelines([r"p.   save.save_reconstructions = false;                      % save reconstructed object and probe when full reconstruction is finished ", "\n"])
        MatlabScript.writelines([r"p.   save.output_file = 'h5';                               % data type of reconstruction file; 'h5' or 'mat'", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"%% Step 9: initialize reconstruction parameters %%%%%%%%%%%%%%%%%%%%", "\n"])
        MatlabScript.writelines([r"% --------- GPU engines  -------------   See for more details: Odstr?il M, et al., Optics express. 2018 Feb 5;26(3):3108-23.", "\n"])
        MatlabScript.writelines([r"eng = struct();                        % reset settings for this engine", "\n"])
        MatlabScript.writelines([r"eng. name = 'GPU';    ", "\n"])
        MatlabScript.writelines([r"eng. use_gpu = true;                   % if false, run CPU code, but it will get very slow ", "\n"])
        MatlabScript.writelines([r"eng. keep_on_gpu = true;               % keep data + projections on GPU, false is useful for large data if DM is used", "\n"])
        MatlabScript.writelines([r"eng. compress_data = false;             % use automatic online memory compression to limit need of GPU memory", "\n"])
        MatlabScript.writelines([r"eng. gpu_id = [];                      % default GPU id, [] means choosen by matlab", "\n"])
        MatlabScript.writelines([r"eng. check_gpu_load = true;            % check available GPU memory before starting GPU engines ", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"% general", "\n"])
        MatlabScript.writelines([r"eng. number_iterations = Niter;          % number of iterations for selected method ", "\n"])
        MatlabScript.writelines([r"eng. asize_presolve = [];      % crop data to 'asize_presolve' size to get low resolution estimate that can be used in the next engine as a good initial guess ", "\n"])
        MatlabScript.writelines([r"eng. align_shared_objects = false;     % before merging multiple unshared objects into one shared, the object will be aligned and the probes shifted by the same distance -> use for alignement and shared reconstruction of drifting scans  ", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"eng. method = GPU_solver;                   % choose GPU solver: DM, ePIE, hPIE, MLc, Mls, -- recommended are MLc and MLs", "\n"])
        MatlabScript.writelines([r"eng. opt_errmetric = errmetric;            % optimization likelihood - poisson, L1", "\n"])
        MatlabScript.writelines([r"eng. grouping = grouping;                    % size of processed blocks, larger blocks need more memory but they use GPU more effeciently, !!! grouping == inf means use as large as possible to fit into memory ", "\n"])
        MatlabScript.writelines([r"                                       % * for hPIE, ePIE, MLs methods smaller blocks lead to faster convergence, ", "\n"])
        MatlabScript.writelines([r"                                       % * for MLc the convergence is similar ", "\n"])
        MatlabScript.writelines([r"                                       % * for DM is has no effect on convergence", "\n"])
        MatlabScript.writelines([r"eng. probe_modes  = p.probe_modes;                % Number of coherent modes for probe", "\n"])
        MatlabScript.writelines([r"eng. object_change_start = object_change_start;          % Start updating object at this iteration number", "\n"])
        MatlabScript.writelines([r"eng. probe_change_start = probe_change_start;           % Start updating probe at this iteration number", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"% regularizations", "\n"])
        MatlabScript.writelines([r"eng. reg_mu = 0;                       % Regularization (smooting) constant ( reg_mu = 0 for no regularization)", "\n"])
        MatlabScript.writelines([r"eng. delta = 0;                        % press values to zero out of the illumination area in th object, usually 1e-2 is enough ", "\n"])
        MatlabScript.writelines([r"eng. positivity_constraint_object = 0; % enforce weak (relaxed) positivity in object, ie O = O*(1-a)+a*|O|, usually a=1e-2 is already enough. Useful in conbination with OPRP or probe_fourier_shift_search  ", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"eng. apply_multimodal_update = true; % apply all incoherent modes to object, it can cause isses if the modes collect some crap ", "\n"])
        MatlabScript.writelines([r"eng. probe_backpropagate = 0;         % backpropagation distance the probe mask, 0 == apply in the object plane. Useful for pinhole imaging where the support can be applied  at the pinhole plane", "\n"])
        MatlabScript.writelines([r"eng. probe_support_radius = [];       % Normalized radius of circular support, = 1 for radius touching the window    ", "\n"])
        MatlabScript.writelines([r"eng. probe_support_fft = false;       % assume that there is not illumination intensity out of the central FZP cone and enforce this contraint. Useful for imaging with focusing optics. Helps to remove issues from the gaps between detector modules.", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"% basic recontruction parameters ", "\n"])
        MatlabScript.writelines([r"% PIE / ML methods                    % See for more details: Odstr?il M, et al., Optics express. 2018 Feb 5;26(3):3108-23.", "\n"])
        MatlabScript.writelines([r"eng. beta_object = 1;                 % object step size, larger == faster convergence, smaller == more robust, should not exceed 1", "\n"])
        MatlabScript.writelines([r"eng. beta_probe = 1;                  % probe step size, larger == faster convergence, smaller == more robust, should not exceed 1", "\n"])
        MatlabScript.writelines([r"eng. delta_p = 0.1;                   % LSQ dumping constant, 0 == no preconditioner, 0.1 is usually safe, Preconditioner accelerates convergence and ML methods become approximations of the second order solvers ", "\n"])
        MatlabScript.writelines([r"eng. momentum = 0;                    % add momentum acceleration term to the MLc method, useful if the probe guess is very poor or for acceleration of multilayer solver, but it is quite computationally expensive to be used in conventional ptycho without any refinement. ", "\n"])
        MatlabScript.writelines([r"                                      % The momentum method works usually well even with the accelerated_gradients option.  eng.momentum = multiplication gain for velocity, eng.momentum == 0 -> no acceleration, eng.momentum == 0.5 is a good value", "\n"])
        MatlabScript.writelines([r"                                      % momentum is enabled only when par.Niter < par.accelerated_gradients_start;", "\n"])
        MatlabScript.writelines([r"eng. accelerated_gradients_start = inf; % iteration number from which the Nesterov gradient acceleration should be applied, this option is supported only for MLc method. It is very computationally cheap way of convergence acceleration. ", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"% DM", "\n"])
        MatlabScript.writelines([r"eng. pfft_relaxation = 0.05;          % Relaxation in the Fourier domain projection, = 0  for full projection ", "\n"])
        MatlabScript.writelines([r"eng. probe_regularization = 0.1;      % Weight factor for the probe update (inertia)", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"% ADVANCED OPTIONS                     See for more details: Odstr?il M, et al., Optics express. 2018 Feb 5;26(3):3108-23.", "\n"])
        MatlabScript.writelines([r"% position refinement ", "\n"])
        MatlabScript.writelines([r"eng. apply_subpix_shift = true;       % apply FFT-based subpixel shift, it is automatically allowed for position refinement", "\n"])
        MatlabScript.writelines([r"eng. probe_position_search = N_pos_corr;      % iteration number from which the engine will reconstruct probe positions, from iteration == probe_position_search, assume they have to match geometry model with error less than probe_position_error_max", "\n"])
        MatlabScript.writelines([r"eng. probe_geometry_model = {'scale', 'asymmetry', 'rotation', 'shear'};  % list of free parameters in the geometry model, choose from: {'scale', 'asymmetry', 'rotation', 'shear'}", "\n"])
        MatlabScript.writelines([r"%eng. probe_geometry_model = {};  % list of free parameters in the geometry model, choose from: {'scale', 'asymmetry', 'rotation', 'shear'}", "\n"])
        MatlabScript.writelines([r"eng. probe_position_error_max = inf; % maximal expected random position errors, probe prositions are confined in a circle with radius defined by probe_position_error_max and with center defined by original positions scaled by probe_geometry_model", "\n"])
        MatlabScript.writelines([r"eng. apply_relaxed_position_constraint = false; % added by YJ. Apply a relaxed constraint to probe positions. default = true. Set to false if there are big jumps in positions.", "\n"])
        MatlabScript.writelines([r"eng. update_pos_weight_every = 100; % added by YJ. Allow position weight to be updated multiple times. default = inf: only update once.", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"% multilayer extension ", "\n"])
        MatlabScript.writelines([r"eng. delta_z = [];                     % if not empty, use multilayer ptycho extension , see ML_MS code for example of use, [] == common single layer ptychography , note that delta_z provides only relative propagation distance from the previous layer, ie delta_z can be either positive or negative. If preshift_ML_probe == false, the first layer is defined by position of initial probe plane. It is useful to use eng.momentum for convergence acceleration ", "\n"])
        MatlabScript.writelines([r"eng. regularize_layers = 0;            % multilayer extension: 0<R<<1 -> apply regularization on the reconstructed object layers, 0 == no regularization, 0.01 == weak regularization that will slowly symmetrize information content between layers ", "\n"])
        MatlabScript.writelines([r"eng. preshift_ML_probe = true;         % multilayer extension: if true, assume that the provided probe is reconstructed in center of the sample and the layers are centered around this position ", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"% other extensions ", "\n"])
        MatlabScript.writelines([r"eng. background = 0;                   % average background scattering level, for OMNI values around 0.3 for 100ms, for flOMNI <0.1 per 100ms exposure, see for more details: Odstrcil, M., et al., Optics letters 40.23 (2015): 5574-5577.", "\n"])
        MatlabScript.writelines([r"eng. background_width = inf;           % width of the background function in pixels,  inf == flat background, background function is then convolved with the average diffraction pattern in order to account for beam diversion ", "\n"])
        MatlabScript.writelines([r"eng. clean_residua = false;            % remove phase residua from reconstruction by iterative unwrapping, it will result in low spatial freq. artefacts -> object can be used as an residua-free initial guess for netx engine", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"% wavefront & camera geometry refinement     See for more details: Odstrcil M, et al., Optics express. 2018 Feb 5;26(3):3108-23.", "\n"])
        MatlabScript.writelines([r"eng. probe_fourier_shift_search = inf; % iteration number from which the engine will: refine farfield position of the beam (ie angle) from iteration == probe_fourier_shift_search", "\n"])
        MatlabScript.writelines([r"eng. estimate_NF_distance = inf;       % iteration number from which the engine will: try to estimate the nearfield propagation distance using gradient descent optimization  ", "\n"])
        MatlabScript.writelines([r"eng. detector_rotation_search = inf;   % iteration number from which the engine will: search for optimal detector rotation, preferably use with option mirror_scan = true , rotation of the detector axis with respect to the sample axis, similar as rotation option in the position refinement geometry model but works also for 0/180deg rotation shared scans ", "\n"])
        MatlabScript.writelines([r"eng. detector_scale_search = inf;      % iteration number from which the engine will: refine pixel scale of the detector, can be used to refine propagation distance in ptycho ", "\n"])
        MatlabScript.writelines([r"eng. variable_probe = variable_probe_modes>0;           % Use SVD to account for variable illumination during a single (coupled) scan, see for more details:  Odstrcil, M. et al. Optics express 24.8 (2016): 8360-8369.", "\n"])
        MatlabScript.writelines([r"eng. variable_probe_modes = variable_probe_modes;         % OPRP settings , number of SVD modes using to describe the probe evolution. ", "\n"])
        MatlabScript.writelines([r"eng. variable_probe_smooth = 0;        % OPRP settings , enforce of smooth evolution of the OPRP modes -> N is order of polynomial fit used for smoothing, 0 == do not apply any smoothing. Smoothing is useful if only a smooth drift is assumed during the ptycho acquisition ", "\n"])
        MatlabScript.writelines([r"eng. variable_intensity = variable_probe_modes>0;       % account to changes in probe intensity", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"% extra analysis", "\n"])
        MatlabScript.writelines([r"eng. get_fsc_score = false;            % measure evolution of the Fourier ring correlation during convergence ", "\n"])
        MatlabScript.writelines([r"eng. mirror_objects = false;           % mirror objects, useful for 0/180deg scan sharing -> geometry refinement for tomography, works only if 2 scans are provided ", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"% custom data adjustments, useful for offaxis ptychography", "\n"])
        MatlabScript.writelines([r"eng.auto_center_data = false;           % autoestimate the center of mass from data and shift the diffraction patterns so that the average center of mass corresponds to center of mass of the provided probe ", "\n"])
        MatlabScript.writelines([r"eng.auto_center_probe = false;          % center the probe position in real space before reconstruction is started ", "\n"])
        MatlabScript.writelines([r"eng.custom_data_flip = [0,1,1];         % apply custom flip of the data [fliplr, flipud, transpose]  - can be used for quick testing of reconstruction with various flips or for reflection ptychography ", "\n"])
        MatlabScript.writelines([r"eng.apply_tilted_plane_correction = ''; % if any(p.sample_rotation_angles([1,2]) ~= 0),  this option will apply tilted plane correction. (a) 'diffraction' apply correction into the data, note that it is valid only for 'low NA' illumination  Gardner, D. et al., Optics express 20.17 (2012): 19050-19059. (b) 'propagation' - use tilted plane propagation, (c) '' - will not apply any correction ", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"% I/O", "\n"])
        MatlabScript.writelines([r"eng.plot_results_every = Niter_plot_results;", "\n"])
        MatlabScript.writelines([r"eng.save_results_every = Niter_save_results;", "\n"])
        MatlabScript.writelines([r"eng.save_images ={'obj_ph','probe_mag','probe'};", "\n"])
        MatlabScript.writelines([r"eng.extraPrintInfo = data_descriptor;", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"resultDir = strcat(p.base_path, sprintf(p.scan.format, p.scan_number),'/roi',p.scan.roi_label,'/');", "\n"])
        MatlabScript.writelines([r"[eng.fout, p.suffix] = generateResultDir(eng, resultDir);", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"%add engine", "\n"])
        MatlabScript.writelines([r"[p, ~] = core.append_engine(p, eng);    % Adds this engine to the reconstruction process", "\n"])
        MatlabScript.writelines([r"", "\n"])
        MatlabScript.writelines([r"%% Step 10: Run the reconstruction", "\n"])
        MatlabScript.writelines([r"tic", "\n"])
        MatlabScript.writelines([r"out = core.ptycho_recons(p);", "\n"])
        MatlabScript.writelines([r"toc", "\n"])
        MatlabScript.writelines([r"", "\n"])

## 2. Slurm sbatch script

In [8]:
# Setup
idx_offset = 11 #1

for ii in range(len(scan_folder_list)//2):
    MatlabScriptName_01 = f'runPtycho_cc_Themis_{ExpDate}_{Sample}_{ExpTime_list[2*ii]}'
    MatlabScriptName_02 = f'runPtycho_cc_Themis_{ExpDate}_{Sample}_{ExpTime_list[2*ii+1]}'
    name_sbatch = f'4hr_2task_V100_runPtycho_{ii+idx_offset:02}.sbatch'
    jobName = f'{ExpDate}_{ii+idx_offset:02}'
    with open (os.path.join(output_dir, name_sbatch), 'w', newline='\n') as sbatch:  
        sbatch.writelines([r'#!/bin/bash', '\n'])
        sbatch.writelines([r'###############################################################################', '\n'])
        sbatch.writelines([r'#', '\n'])
        sbatch.writelines([r'#SBATCH --job-name=', jobName, r'                         # Name of batch job, Change this!', '\n'])
        sbatch.writelines([r'#SBATCH --partition=secondary-eth,eng-research-gpu # Partition (queue), eng-research-gpu has 7 V100(16GB)x2 nodes; secondary-eth has eth has 3 V100(32GB)x2 node and 1 P100(12)x4 node', '\n'])
        sbatch.writelines([r'#SBATCH --time=04:00:00                            # Job run time (hh:mm:ss)', '\n'])
        sbatch.writelines([r'#SBATCH --nodes=1                                  # Number of nodes', '\n'])
        sbatch.writelines([r'#SBATCH --ntasks=2                                 # Number of task (cores/ppn) per node, one task per ptycho recon', '\n'])
        sbatch.writelines([r'#SBATCH --cpus-per-task=4                          # Always at least twice the number of GPUs (Four times the number for TITAN and V100)', '\n'])
        sbatch.writelines([r"#SBATCH --gpus-per-task=V100:1                     # Number of gpus per task, specifying V100 is helpful because it's almost 2x faster than P100", '\n'])
        sbatch.writelines([r'#SBATCH --mem-per-gpu=80GB                         # Specify the needed memory, the default is unlimited but for some reason eng nodes are only requested with 37.5GB and secondary-eth with 114GB', '\n'])
        sbatch.writelines([r'#SBATCH --output=log_job_%x_4hr_%A.txt             # Name of batch job output file', '\n'])
        sbatch.writelines([r'##SBATCH --error=err_%x_4hr_%A.txt                 # Name of batch job error file', '\n'])
        sbatch.writelines([r'##SBATCH --mail-user=NetID@illinois.edu            # Send email notifications!/bin/bash', '\n'])
        sbatch.writelines([r'##SBATCH --mail-type=BEGIN,END                     # Type of email notifications to send', '\n'])
        sbatch.writelines([r'#', '\n'])
        sbatch.writelines([r'# UPDATED: 2022/12/13 Chia-Hao Lee, Slurm 22.05', '\n'])
        sbatch.writelines([r'###############################################################################', '\n'])
        sbatch.writelines([r'', '\n'])
        sbatch.writelines([r'# Clear the value set in the DISPLAY environment variable', '\n'])
        sbatch.writelines([r'# to run the CLI version of MATLAB', '\n'])
        sbatch.writelines([r'unset DISPLAY', '\n'])
        sbatch.writelines([r'', '\n'])
        sbatch.writelines([r'# Load MATLAB module (Enable MATLAB in user environment)', '\n'])
        sbatch.writelines([r'module load matlab/9.7 # Only this version has the correct path to nvcc compiler on eng-research-gpu, eth, fdr', '\n'])
        sbatch.writelines([r'', '\n'])
        sbatch.writelines([r'echo "Starting job $SLURM_JOB_ID on $HOSTNAME"', '\n'])
        sbatch.writelines([r'', '\n'])
        sbatch.writelines([r'# Start 2 tasks concurrently, each take 4 CPU and 1 GPU ', '\n'])
        sbatch.writelines([r'# Note that each node may has different amount of memory', '\n'])
        sbatch.writelines([r"srun --ntasks=1 --cpus-per-task=4 --gpu-bind=single:1 bash -c 'nvidia-smi -L; matlab -logfile ", f'log_{MatlabScriptName_01}', r'.txt -r "', MatlabScriptName_01, r'''"' &''', '\n'])
        sbatch.writelines([r"srun --ntasks=1 --cpus-per-task=4 --gpu-bind=single:1 bash -c 'nvidia-smi -L; matlab -logfile ", f'log_{MatlabScriptName_02}', r'.txt -r "', MatlabScriptName_02, r'''"' &''', '\n'])
        sbatch.writelines([r'wait', '\n'])
        sbatch.writelines([r'', '\n'])
        sbatch.writelines([r"# Don't use dash (-) in the <matlab_script_name> because '-r' will fail to recognize the script name.", '\n'])
        sbatch.writelines([r"# The '.m' file extension is not needed for <matlab_script_name> while using '-r'", '\n'])


    print(f'4hr_2task_V100_runPtycho_{ii+idx_offset:02}.sbatch has been generated!') 

4hr_2task_V100_runPtycho_11.sbatch has been generated!
4hr_2task_V100_runPtycho_12.sbatch has been generated!
4hr_2task_V100_runPtycho_13.sbatch has been generated!
4hr_2task_V100_runPtycho_14.sbatch has been generated!
4hr_2task_V100_runPtycho_15.sbatch has been generated!
4hr_2task_V100_runPtycho_16.sbatch has been generated!
4hr_2task_V100_runPtycho_17.sbatch has been generated!
4hr_2task_V100_runPtycho_18.sbatch has been generated!
4hr_2task_V100_runPtycho_19.sbatch has been generated!
4hr_2task_V100_runPtycho_20.sbatch has been generated!


## 3. BatchSubmit.sh

In [14]:
# Setup
num_sbatch = [idx_offset, len(scan_folder_list)//2 + idx_offset - 1]
print(num_sbatch)

with open (os.path.join(output_dir, 'BatchSubmit.sh'), 'w', newline='\n') as BatchSubmit:  
    BatchSubmit.writelines([r'for i in $(seq -f "%02g" ', f'{num_sbatch[0]} {num_sbatch[1]}); do', '\n'])
    BatchSubmit.writelines([r'echo Submitting 4hr_2task_V100_runPtycho_${i}.sbatch', '\n'])
    BatchSubmit.writelines([r'sbatch 4hr_2task_V100_runPtycho_${i}.sbatch', '\n'])
    BatchSubmit.writelines([r'sleep 1', '\n'])
    BatchSubmit.writelines([r'done', '\n'])
    BatchSubmit.writelines([r'echo "Jobs are all submitted!"', '\n'])
print('BatchSubmit.sh has been generated!')

[11, 20]
BatchSubmit.sh has been generated!
